In [1]:
from colorama import (Fore, Style)

from collections import (ChainMap, OrderedDict)
from collections.abc import Iterable
from pprint import (pformat, pprint)

import itertools
import types

In [2]:
rwb = ('r', 'w', 'b')
oyg = ('o', 'y', 'g')

console_colors = {
    'r': Fore.RED,
    'w': Fore.WHITE,
    'b': Fore.BLUE,
    'o': Fore.LIGHTYELLOW_EX,
    'y': Fore.YELLOW,
    'g': Fore.GREEN,
}

colorstrs = {
    'r': f'red',
    'w': f'white',
    'b': f'blue',
    'o': f'orange',
    'y': f'yellow',
    'g': f'green',
}

short_colorstrs = { k:f'{console_colors[k]}{k}{Style.RESET_ALL}' for k, v in colorstrs.items() }
long_colorstrs  = { k:f'{console_colors[k]}{v}{Style.RESET_ALL}' for k, v in colorstrs.items() }

assert len(rwb) == len(set(rwb))
assert len(oyg) == len(set(oyg))
assert len(rwb) == len(oyg)

In [3]:
colors = frozenset(rwb).union(frozenset(oyg))

anticolors = ChainMap(
    dict(zip(rwb, oyg)),
    dict(zip(oyg, rwb))
)

assert len(colors) == len(anticolors)

print(f'colors:     {colors}')
print(f'anticolors: {anticolors}')
print(f'anticolors: { {**anticolors}}')

colors:     frozenset({'g', 'r', 'b', 'o', 'w', 'y'})
anticolors: ChainMap({'r': 'o', 'w': 'y', 'b': 'g'}, {'o': 'r', 'y': 'w', 'g': 'b'})
anticolors: {'o': 'r', 'y': 'w', 'g': 'b', 'r': 'o', 'w': 'y', 'b': 'g'}


In [4]:
def unescape(string):
    return string.encode('utf-8').decode('unicode_escape')

In [5]:
def tupleify(item):
    if isinstance(item, (filter, map, types.GeneratorType)): return tuple(item)
    return item

In [6]:
def forward(order):
    order = tupleify(order)
    return order[-1:]+order[:-1]

def backward(order):
    order = tupleify(order)
    return order[1:]+order[:1]

def make_anti(thing):
    thing = tupleify(thing)

    if isinstance(thing, str):
        return anticolors[thing]
    elif isinstance(thing, dict):
        return {k:anticolors[v] for k, v in thing.items()}
    elif isinstance(thing, (set, frozenset)):
        return frozenset((k,anticolors[v]) for k,v in thing)
    if isinstance(thing, tuple) and len(thing[0]) == 2:
        return tuple((k,anticolors[v]) for k, v in thing)

    return type(thing)(map(make_anti, thing))

In [7]:
def get_adjacents(color):
    return filter(lambda c: c != color and c != anticolors[color], colors)

In [8]:
def is_anti(cube1, cube2):
    for k1, v1 in cube1:
        for k2, v2 in filter(lambda kv2: kv2[0] == k1, cube2):
            if not v1 == anticolors[v2]:
                return False
    return True

def is_solved(cube):
    cube = tupleify(cube)
    return len(cube) == sum(1 for _ in filter(lambda kv: kv[0] == kv[1], cube))
    
def is_not_solved(cube):
    return not is_solved(cube)

def is_anti_solved(cube):
    return is_solved(make_anti(cube))

def is_not_anti_solved(cube):
    return not is_anti_solved(cube)

def is_ptn(cube, ptn=rwb):
    cube = tupleify(cube)
    ant = tuple(filter(lambda c: c not in ptn, colors))
    for k, v in cube:
        if k in ptn and v not in ptn: return False
        if k in ant and v not in ant: return False
    return True

def is_rwb(cube):
    return is_ptn(cube, ptn=rwb)

In [9]:
def get_pairs(cube, useequal=True):
    return frozenset(
        {frozenset({k, v}) for k, v in cube if useequal or k != v}
    )

def get_npairs(cube, **kwargs):
    return len(get_pairs(cube, **kwargs))

In [10]:
def filter_colors(*args, colors=colors):
    return colors.difference({*args, *map(make_anti, args)})

def make_rots(colors=colors, anticolors=anticolors):
    return frozenset({
        frozenset({(c1, c2), (c2, c3), (c3, c1)})
            for c1 in colors
                for c2 in filter_colors(c1)
                    for c3 in filter_colors(c1, c2)
    })

In [11]:
def to_sorted(thing, reverse=False):
    if     isinstance(thing, str):      return thing
    if not isinstance(thing, Iterable): return thing

    # assume its a mapping not meant to be sorted
    if isinstance(thing, tuple) and len(thing) == 2:
        return thing

    sthing = sorted(map(to_sorted, thing), reverse=reverse)

    if isinstance(thing, (frozenset, set, dict, map, filter)):
        return tuple(sthing)

    return type(thing)(sthing)

In [12]:
def to_frozen(thing):
    if isinstance(thing, (str, dict)):  return thing
    if not isinstance(thing, Iterable): return thing

    if isinstance(thing, tuple) and len(thing) == 2:
        return thing

    return frozenset(map(to_frozen, thing))

In [13]:
def to_set(thing):
    if isinstance(thing, (str, dict)):  return thing
    if not isinstance(thing, Iterable): return thing

    if isinstance(thing, tuple) and len(thing) == 2:
        return thing

    return set(map(to_frozen, thing))

In [14]:
def string_swap(item, colorstrs=short_colorstrs):    
    if isinstance(item, str):
        if item not in colorstrs:
            return item
        width = max(map(len, colorstrs.values()))
        return f'{colorstrs[item]:{width}}'

    if not isinstance(item, Iterable):
        return item

    return type(item)(map(lambda s: string_swap(s, colorstrs=colorstrs), item))

In [15]:
def unescape_pformat(item, colorstrs=short_colorstrs, width=400, **kwargs):
    item = tupleify(item)
    kwargs.update(width=width)
    return unescape(
        pformat(string_swap(item, colorstrs=colorstrs), **kwargs)
    )

def unescape_pprint(item, **kwargs):
    print(unescape_pformat(item, **kwargs))

In [16]:
def remove_anti(checkers, reverse=False):
    checkers = tupleify(checkers)
    scheckers = set()
    for chk in checkers:
        fcheckers = list(filter(lambda chk2: chk2 is chk or is_anti(chk2, chk), checkers))

        scheckers.add(
            fcheckers[-1 if reverse else 0]
        )
    return frozenset(scheckers)

In [17]:
def str_cube(cube, sort=True, reverse=False, width=None, sep=', ', colorstrs=long_colorstrs):
    cube = tupleify(cube)

    if sort: cube = to_sorted(cube, reverse=reverse)

    if colorstrs is not None:
        cube = ( (colorstrs[k], colorstrs[v]) for k, v in cube )

    if width is None:
        if colorstrs is None:
            width = max(map(len, itertools.chain(*zip(*cube))))
        else:
            width = max(map(len, colorstrs.values()))

    return sep.join(f'{k:{width}} => {v:{width}}' for k, v in cube)

def print_cube(cube, **kwargs):
    print(str_cube(cube, **kwargs))

In [18]:
def str_cubes(cubes, sort=True, reverse=False, cubesep='\n', **kwargs):
    cubes = tupleify(cubes)
    if sort: cubes = to_sorted(cubes, reverse=reverse)    
    kwargs.update({'sort':sort, 'reverse': reverse})
    return f'ncubes: {len(cubes)}\n'+cubesep.join(map(lambda cube: str_cube(cube, **kwargs), cubes))

def print_cubes(cubes, **kwargs):
    print(str_cubes(cubes, **kwargs))

In [19]:
rots = make_rots()
print(f'nrotations: {len(rots)}')
unescape_pprint(to_sorted(rots), colorstrs=long_colorstrs)

nrotations: 16
((('blue  ', 'orange'), ('orange', 'white '), ('white ', 'blue  ')),
 (('blue  ', 'orange'), ('orange', 'yellow'), ('yellow', 'blue  ')),
 (('blue  ', 'red   '), ('red   ', 'white '), ('white ', 'blue  ')),
 (('blue  ', 'red   '), ('red   ', 'yellow'), ('yellow', 'blue  ')),
 (('blue  ', 'white '), ('orange', 'blue  '), ('white ', 'orange')),
 (('blue  ', 'white '), ('red   ', 'blue  '), ('white ', 'red   ')),
 (('blue  ', 'yellow'), ('orange', 'blue  '), ('yellow', 'orange')),
 (('blue  ', 'yellow'), ('red   ', 'blue  '), ('yellow', 'red   ')),
 (('green ', 'orange'), ('orange', 'white '), ('white ', 'green ')),
 (('green ', 'orange'), ('orange', 'yellow'), ('yellow', 'green ')),
 (('green ', 'red   '), ('red   ', 'white '), ('white ', 'green ')),
 (('green ', 'red   '), ('red   ', 'yellow'), ('yellow', 'green ')),
 (('green ', 'white '), ('orange', 'green '), ('white ', 'orange')),
 (('green ', 'white '), ('red   ', 'green '), ('white ', 'red   ')),
 (('green ', 'yello

In [20]:
def rot_to_order(rot):
    if not isinstance(rot, dict):
        rot = dict(rot)

    order = []
    color = next(iter(rot))
    for _ in range(len(rot)):
        order.append(color)
        color = rot[color]

    return tuple(order)

def make_checkers(rots):
    solved   = frozenset(zip(rwb, rwb)).union(zip(make_anti(rwb), make_anti(rwb)))
    checkers = set({solved, make_anti(solved)})

    for rot in rots:
        order     = rot_to_order(rot)
        antiorder = make_anti(order)

        chk     = frozenset(zip(order, forward( order))).union(zip(antiorder, forward( antiorder)))
        bck_chk = frozenset(zip(order, backward(order))).union(zip(antiorder, backward(antiorder)))

        assert len(chk)     == len(colors), (chk, colors)
        assert len(bck_chk) == len(colors), (bck_chk, colors)

        checkers.update({
            chk,
            make_anti(chk),
            bck_chk,
            make_anti(bck_chk)
        })

    return frozenset(checkers)

In [21]:
checkers          = make_checkers(rots)
fwd_anti          = remove_anti(checkers, False)
bck_anti          = remove_anti(checkers, True )
no_anti_checkers  = bck_anti

assert no_anti_checkers.issubset(checkers)
assert checkers.issuperset(no_anti_checkers)

assert fwd_anti != bck_anti
assert set() == fwd_anti.intersection(bck_anti)
assert fwd_anti.union(               bck_anti) == checkers
assert fwd_anti.symmetric_difference(bck_anti) == checkers

chkint = checkers.intersection(no_anti_checkers)
chkdif = checkers.difference(  no_anti_checkers)

assert set() == no_anti_checkers.difference(checkers)
assert checkers == chkdif.symmetric_difference(chkint)

print(f'num:                                {len(checkers)}')
print(f'num not solved:                     {sum(1 for i in filter(is_not_solved, checkers))}')
print(f'num not solved and not anti-solved: {sum(1 for i in filter(is_not_anti_solved, filter(is_not_solved, checkers)))}')
print(f'num rwb:                            {sum(1 for i in filter(is_rwb, checkers))}')

num:                                18
num not solved:                     17
num not solved and not anti-solved: 16
num rwb:                            3


In [22]:
def check_cube(chk):
    solved      = is_solved(chk)
    anti_solved = is_anti_solved(chk)

    chk = dict(chk)
    
    for k, v in chk.items():
        if not solved:
            assert k != v, chk
        if not anti_solved:
            assert v != anticolors[k], chk
            assert k != anticolors[v], chk

        vadj = tuple(get_adjacents(v))

        for a in get_adjacents(k):
            av = chk[a]
            assert av in vadj, (chk, (k, v, a, av), tuple(vadj))
            assert v in get_adjacents(av), tuple(get_adjacents(av))

def check_cubes(checkers):
    checkers = tupleify(checkers)

    equivalents = map(
        lambda chk: sum(
            1 for _ in filter(lambda chk2: chk2 == chk, checkers)
        ),
        checkers
    )

    assert sum(equivalents) == len(checkers)

    for chk in checkers:
        matches = filter(lambda a: a == chk, checkers)
        assert 1 == sum(1 for _ in matches)
    return tuple(map(check_cube, checkers))

In [23]:
check_cubes(checkers)
check_cubes(to_sorted(checkers))

assert len(to_sorted(checkers)) == len(checkers)
assert len(frozenset(to_sorted(checkers)))  == len(checkers)

In [24]:
print_cubes(filter(is_solved, checkers))

ncubes: 1
blue   => blue  , green  => green , orange => orange, red    => red   , white  => white , yellow => yellow


In [25]:
print_cubes(filter(is_anti_solved, checkers))

ncubes: 1
blue   => green , green  => blue  , orange => red   , red    => orange, white  => yellow, yellow => white 


In [26]:
print_cubes(filter(is_rwb, checkers))

ncubes: 3
blue   => blue  , green  => green , orange => orange, red    => red   , white  => white , yellow => yellow
blue   => red   , green  => orange, orange => yellow, red    => white , white  => blue  , yellow => green 
blue   => white , green  => yellow, orange => green , red    => blue  , white  => red   , yellow => orange


In [27]:
print_cubes(filter(is_not_solved, filter(is_not_anti_solved, no_anti_checkers)))

ncubes: 8
blue   => orange, green  => red   , orange => white , red    => yellow, white  => green , yellow => blue  
blue   => orange, green  => red   , orange => yellow, red    => white , white  => green , yellow => blue  
blue   => red   , green  => orange, orange => white , red    => yellow, white  => green , yellow => blue  
blue   => red   , green  => orange, orange => yellow, red    => white , white  => green , yellow => blue  
blue   => white , green  => yellow, orange => blue  , red    => green , white  => red   , yellow => orange
blue   => white , green  => yellow, orange => green , red    => blue  , white  => orange, yellow => red   
blue   => yellow, green  => white , orange => blue  , red    => green , white  => orange, yellow => red   
blue   => yellow, green  => white , orange => green , red    => blue  , white  => red   , yellow => orange


In [28]:
print_cubes(no_anti_checkers)

ncubes: 9
blue   => blue  , green  => green , orange => orange, red    => red   , white  => white , yellow => yellow
blue   => orange, green  => red   , orange => white , red    => yellow, white  => green , yellow => blue  
blue   => orange, green  => red   , orange => yellow, red    => white , white  => green , yellow => blue  
blue   => red   , green  => orange, orange => white , red    => yellow, white  => green , yellow => blue  
blue   => red   , green  => orange, orange => yellow, red    => white , white  => green , yellow => blue  
blue   => white , green  => yellow, orange => blue  , red    => green , white  => red   , yellow => orange
blue   => white , green  => yellow, orange => green , red    => blue  , white  => orange, yellow => red   
blue   => yellow, green  => white , orange => blue  , red    => green , white  => orange, yellow => red   
blue   => yellow, green  => white , orange => green , red    => blue  , white  => red   , yellow => orange


In [29]:
print_cubes(filter(is_not_solved, filter(is_not_anti_solved, checkers)))

ncubes: 16
blue   => orange, green  => red   , orange => white , red    => yellow, white  => blue  , yellow => green 
blue   => orange, green  => red   , orange => white , red    => yellow, white  => green , yellow => blue  
blue   => orange, green  => red   , orange => yellow, red    => white , white  => blue  , yellow => green 
blue   => orange, green  => red   , orange => yellow, red    => white , white  => green , yellow => blue  
blue   => red   , green  => orange, orange => white , red    => yellow, white  => blue  , yellow => green 
blue   => red   , green  => orange, orange => white , red    => yellow, white  => green , yellow => blue  
blue   => red   , green  => orange, orange => yellow, red    => white , white  => blue  , yellow => green 
blue   => red   , green  => orange, orange => yellow, red    => white , white  => green , yellow => blue  
blue   => white , green  => yellow, orange => blue  , red    => green , white  => orange, yellow => red   
blue   => white , green  =

In [30]:
print_cubes(checkers)

ncubes: 18
blue   => blue  , green  => green , orange => orange, red    => red   , white  => white , yellow => yellow
blue   => green , green  => blue  , orange => red   , red    => orange, white  => yellow, yellow => white 
blue   => orange, green  => red   , orange => white , red    => yellow, white  => blue  , yellow => green 
blue   => orange, green  => red   , orange => white , red    => yellow, white  => green , yellow => blue  
blue   => orange, green  => red   , orange => yellow, red    => white , white  => blue  , yellow => green 
blue   => orange, green  => red   , orange => yellow, red    => white , white  => green , yellow => blue  
blue   => red   , green  => orange, orange => white , red    => yellow, white  => blue  , yellow => green 
blue   => red   , green  => orange, orange => white , red    => yellow, white  => green , yellow => blue  
blue   => red   , green  => orange, orange => yellow, red    => white , white  => blue  , yellow => green 
blue   => red   , green  =

In [31]:
for cube in no_anti_checkers:
    cubes = (cube, make_anti(cube))
    #cubes = filter(is_not_solved,      cubes)
    #cubes = filter(is_not_anti_solved, cubes)
    print_cubes(cubes)

ncubes: 2
blue   => red   , green  => orange, orange => white , red    => yellow, white  => green , yellow => blue  
blue   => orange, green  => red   , orange => yellow, red    => white , white  => blue  , yellow => green 
ncubes: 2
blue   => white , green  => yellow, orange => blue  , red    => green , white  => red   , yellow => orange
blue   => yellow, green  => white , orange => green , red    => blue  , white  => orange, yellow => red   
ncubes: 2
blue   => yellow, green  => white , orange => blue  , red    => green , white  => orange, yellow => red   
blue   => white , green  => yellow, orange => green , red    => blue  , white  => red   , yellow => orange
ncubes: 2
blue   => white , green  => yellow, orange => green , red    => blue  , white  => orange, yellow => red   
blue   => yellow, green  => white , orange => blue  , red    => green , white  => red   , yellow => orange
ncubes: 2
blue   => yellow, green  => white , orange => green , red    => blue  , white  => red   , yell

In [32]:
assert no_anti_checkers == to_frozen(to_sorted(no_anti_checkers))
unescape_pprint(to_set(no_anti_checkers), colorstrs=long_colorstrs)
unescape_pprint(to_sorted(no_anti_checkers), colorstrs=long_colorstrs)

{frozenset({('yellow', 'red   '), ('orange', 'blue  '), ('white ', 'orange'), ('red   ', 'green '), ('blue  ', 'yellow'), ('green ', 'white ')}),
 frozenset({('blue  ', 'orange'), ('red   ', 'white '), ('white ', 'green '), ('green ', 'red   '), ('orange', 'yellow'), ('yellow', 'blue  ')}),
 frozenset({('green ', 'green '), ('orange', 'orange'), ('blue  ', 'blue  '), ('red   ', 'red   '), ('white ', 'white '), ('yellow', 'yellow')}),
 frozenset({('green ', 'orange'), ('red   ', 'yellow'), ('white ', 'green '), ('orange', 'white '), ('blue  ', 'red   '), ('yellow', 'blue  ')}),
 frozenset({('yellow', 'red   '), ('orange', 'green '), ('white ', 'orange'), ('blue  ', 'white '), ('green ', 'yellow'), ('red   ', 'blue  ')}),
 frozenset({('green ', 'white '), ('orange', 'green '), ('white ', 'red   '), ('blue  ', 'yellow'), ('red   ', 'blue  '), ('yellow', 'orange')}),
 frozenset({('blue  ', 'orange'), ('red   ', 'yellow'), ('white ', 'green '), ('orange', 'white '), ('green ', 'red   '), ('

In [33]:
unescape_pprint(          to_set(map(lambda c: get_npairs(c, useequal=False), checkers)))
unescape_pprint(          to_set(map(lambda c: get_npairs(c, useequal=True ), checkers)))
unescape_pprint(to_sorted(to_set(map(lambda c: get_pairs( c, useequal=True ), checkers))))
unescape_pprint(sorted(map(lambda cube: (f'{hash(get_pairs(cube)):20}', to_sorted(cube)), checkers)), colorstrs=long_colorstrs)

{0, 3, 6}
{3, 6}
((('b',), ('g',), ('o',), ('r',), ('w',), ('y',)),
 (('b', 'g'), ('o', 'r'), ('w', 'y')),
 (('b', 'o'), ('b', 'w'), ('g', 'r'), ('g', 'y'), ('o', 'w'), ('r', 'y')),
 (('b', 'o'), ('b', 'w'), ('g', 'r'), ('g', 'y'), ('o', 'y'), ('r', 'w')),
 (('b', 'o'), ('b', 'y'), ('g', 'r'), ('g', 'w'), ('o', 'w'), ('r', 'y')),
 (('b', 'o'), ('b', 'y'), ('g', 'r'), ('g', 'w'), ('o', 'y'), ('r', 'w')),
 (('b', 'r'), ('b', 'w'), ('g', 'o'), ('g', 'y'), ('o', 'w'), ('r', 'y')),
 (('b', 'r'), ('b', 'w'), ('g', 'o'), ('g', 'y'), ('o', 'y'), ('r', 'w')),
 (('b', 'r'), ('b', 'y'), ('g', 'o'), ('g', 'w'), ('o', 'w'), ('r', 'y')),
 (('b', 'r'), ('b', 'y'), ('g', 'o'), ('g', 'w'), ('o', 'y'), ('r', 'w')))
[(' -761145211498135350', (('blue  ', 'red   '), ('green ', 'orange'), ('orange', 'yellow'), ('red   ', 'white '), ('white ', 'blue  '), ('yellow', 'green '))),
 (' -761145211498135350', (('blue  ', 'white '), ('green ', 'yellow'), ('orange', 'green '), ('red   ', 'blue  '), ('white ', 'red  